## APSRTC

In [31]:
pip install selenium

In [29]:
import pandas as pd

In [32]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

# Click on the 'Bus Operator' selection option
try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button)  # Scroll into view
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

# Click on the "APSRTC" option
try:
    apsrtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='APSRTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", apsrtc_option)  # Scroll into view
    time.sleep(1)
    apsrtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", apsrtc_option)
    time.sleep(3)

# Function to scroll the page dynamically
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

# Collect links from the first page
links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

# Paginate and scrape additional links
for page_number in range(1, 6):
    if page_number < 6:  # Don't try to click next on the last page
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

APSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        APSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

# Iterate over each link to gather bus information
for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in APSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (50, 12). Other element would receive the click: <div class="rb_header " style="height: 100px; display: flex; align-items: center; max-width: 1200px; margin: auto;">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVer

## KERALA RTC

In [42]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

# Click on the 'Bus Operator' selection option
try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button)  # Scroll into view
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

# Click on the "KERALA RTC" option
try:
    kerala_rtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='KERALA RTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", kerala_rtc_option)  # Scroll into view
    time.sleep(1)
    kerala_rtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", kerala_rtc_option)
    time.sleep(3)

# Function to scroll the page dynamically
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

# Collect links from the first page
links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

# Paginate and scrape additional links
for page_number in range(1, 4):
    if page_number < 4:  # Don't try to click next on the last page
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

KSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        KSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

# Iterate over each link to gather bus information
for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                button.click()
                time.sleep(3)
            except NoSuchElementException:
                continue
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in KSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (79, 12). Other element would receive the click: <div class="rb_header " style="height: 100px; display: flex; align-items: center; max-width: 1200px; margin: auto;">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVer

## TSRTC

In [44]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

# Click on the 'Bus Operator' selection option
try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button)  # Scroll into view
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

# Click on the "TSRTC" option
try:
    tsrtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='TSRTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", tsrtc_option)  # Scroll into view
    time.sleep(1)
    tsrtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", tsrtc_option)
    time.sleep(3)

# Function to scroll the page dynamically
def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

# Collect links from the first page
links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

# Paginate and scrape additional links
for page_number in range(1, 4):
    if page_number < 4:  # Don't try to click next on the last page
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

TSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        TSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

# Iterate over each link to gather bus information
for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in TSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (40, 12). Other element would receive the click: <div class="rb_header " style="height: 100px; display: flex; align-items: center; max-width: 1200px; margin: auto;">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVer

## KTCL

In [49]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    ktcl_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='KTCL']")
    driver.execute_script("arguments[0].scrollIntoView(true);", ktcl_option)  
    time.sleep(1)
    ktcl_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", ktcl_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 5):
    if page_number < 5:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

KTCL = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        KTCL.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in KTCL:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (261, 12). Other element would receive the click: <div class="rb_header " style="height: 100px; display: flex; align-items: center; max-width: 1200px; margin: auto;">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVe

## RSRTC

In [51]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    rsrtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='RSRTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", rsrtc_option)  
    time.sleep(1)
    rsrtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", rsrtc_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 4):
    if page_number < 4:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

RSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        RSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in RSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (775, 12). Other element would receive the click: <div class="rb_header " style="height: 100px; display: flex; align-items: center; max-width: 1200px; margin: auto;">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVe

## NBSTC

In [53]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    nbstc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='NBSTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", nbstc_option)  
    time.sleep(1)
    nbstc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", nbstc_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 6):
    if page_number < 6:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

NBSTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        NBSTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in NBSTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (984, 12). Other element would receive the click: <span class="sc-ksYbfQ nuNba">...</span>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVerifier [0x00007FF64FB92A13+1313971]
	GetHandleVerifier [0x00007FF64FA8DD16+245686]
	(

## BSRTC

In [55]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    bsrtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='BSRTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bsrtc_option)  
    time.sleep(1)
    bsrtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bsrtc_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 5):
    if page_number < 5:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

print(links)

BSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        BSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in BSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (985, 12). Other element would receive the click: <span class="sc-ksYbfQ nuNba">...</span>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVerifier [0x00007FF64FB92A13+1313971]
	GetHandleVerifier [0x00007FF64FA8DD16+245686]
	(

## PEPSU

In [57]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    pepsu_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='PEPSU']")
    driver.execute_script("arguments[0].scrollIntoView(true);", pepsu_option)  
    time.sleep(1)
    pepsu_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", pepsu_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 4):
    if page_number < 4:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break            

print(links)

PEPSU = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        PEPSU.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in PEPSU:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (984, 12). Other element would receive the click: <span class="sc-ksYbfQ nuNba">...</span>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVerifier [0x00007FF64FB92A13+1313971]
	GetHandleVerifier [0x00007FF64FA8DD16+245686]
	(

## WBSTC

In [59]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    wbstc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='WBSTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", wbstc_option)  
    time.sleep(1)
    wbstc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", wbstc_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

'''for page_number in range(1, 3):
    if page_number < 3:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
'''            

print(links)

WBSTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        WBSTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in WBSTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (980, 12). Other element would receive the click: <span class="sc-ksYbfQ nuNba">...</span>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVerifier [0x00007FF64FB92A13+1313971]
	GetHandleVerifier [0x00007FF64FA8DD16+245686]
	(

## JKSTRC

In [61]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

driver = webdriver.Chrome()
actions = ActionChains(driver)
wait = WebDriverWait(driver, 60)

driver.get('https://www.redbus.in/')
time.sleep(3)

driver.execute_script("window.scrollBy(0,500);")
time.sleep(3)

try:
    bus_operator_button = driver.find_element(By.XPATH, "//span[@class='sc-hmzhuo dMsnin']")
    driver.execute_script("arguments[0].scrollIntoView(true);", bus_operator_button) 
    time.sleep(1)
    bus_operator_button.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", bus_operator_button)
    time.sleep(3)

try:
    jksrtc_option = driver.find_element(By.XPATH, "//div[@class='rtcName' and text()='JKSRTC']")
    driver.execute_script("arguments[0].scrollIntoView(true);", jksrtc_option)  
    time.sleep(1)
    jksrtc_option.click()
    time.sleep(3)
except ElementClickInterceptedException as e:
    print(f"ElementClickInterceptedException caught: {e}. Retrying with JavaScript click.")
    driver.execute_script("arguments[0].click();", jksrtc_option)
    time.sleep(3)

def scroll_page():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page()
time.sleep(3)
print('Scroll page end')

links = []
link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
links.extend([element.get_attribute('href') for element in link_elements])

for page_number in range(1, 3):
    if page_number < 3:  
        try:
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)
            next_page_button.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            link_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            links.extend([element.get_attribute('href') for element in link_elements])
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
           

print(links)

JKSRTC = []

def collection():
    bus_details = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@class='clearfix bus-item']")))
    for bus_detail in bus_details:
        try:
            bus_route_name = driver.find_element(By.XPATH, ".//h1[@class='D136_h1']").text
        except NoSuchElementException:
            bus_route_name = "N/A"

        try:
            bus_name = bus_detail.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except NoSuchElementException:
            bus_name = "N/A"

        try:
            bus_type = bus_detail.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except NoSuchElementException:
            bus_type = "N/A"

        try:
            departing_time = bus_detail.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except NoSuchElementException:
            departing_time = "N/A"

        try:
            duration = bus_detail.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except NoSuchElementException:
            duration = "N/A"

        try:
            reaching_time = bus_detail.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except NoSuchElementException:
            reaching_time = "N/A"

        try:
            star_rating = bus_detail.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except NoSuchElementException:
            star_rating = "N/A"

        try:
            price = bus_detail.find_element(By.XPATH, ".//div[@class='seat-fare']").text
        except NoSuchElementException:
            price = "N/A"

        try:
            seat_availability = bus_detail.find_element(By.XPATH, ".//div[@class='seat-left m-top-30']").text
        except NoSuchElementException:
            seat_availability = "N/A"

        JKSRTC.append({
            'bus_route_name': bus_route_name,
            'bus_route_link': link,
            'bus_name': bus_name,
            'bus_type': bus_type,
            'departing_time': departing_time,
            'duration': duration,
            'reaching_time': reaching_time,
            'star_rating': star_rating,
            'price': price,
            'seat_availability': seat_availability
        })

for link in links:
    driver.get(link)
    time.sleep(5)
    print('Getting info from the link: ' + link)
    try:
        button_elements = driver.find_elements(By.XPATH, "//div[@class='button']")
        for button in button_elements:
            try:
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException occurred while clicking button on {link}.")
                continue
            
    except TimeoutException:
        continue

    bus_items_loaded = False
    while not bus_items_loaded:
        old_page = driver.page_source
        scroll_page()
        new_page = driver.page_source
        if old_page == new_page:
            bus_items_loaded = True

    try:
        collection()
    except (NoSuchElementException, TimeoutException):
        continue

for bus in JKSRTC:
    print(bus)

driver.quit()

ElementClickInterceptedException caught: Message: element click intercepted: Element <div class="rtcName">...</div> is not clickable at point (979, 12). Other element would receive the click: <span class="sc-ksYbfQ nuNba">...</span>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF64FA59642+30946]
	(No symbol) [0x00007FF64FA0E3D9]
	(No symbol) [0x00007FF64F906FDA]
	(No symbol) [0x00007FF64F95FEEE]
	(No symbol) [0x00007FF64F95D962]
	(No symbol) [0x00007FF64F95AE7B]
	(No symbol) [0x00007FF64F95A095]
	(No symbol) [0x00007FF64F94C271]
	(No symbol) [0x00007FF64F97CA6A]
	(No symbol) [0x00007FF64F94BBB6]
	(No symbol) [0x00007FF64F97CC80]
	(No symbol) [0x00007FF64F99B041]
	(No symbol) [0x00007FF64F97C813]
	(No symbol) [0x00007FF64F94A6E5]
	(No symbol) [0x00007FF64F94B021]
	GetHandleVerifier [0x00007FF64FB8F84D+1301229]
	GetHandleVerifier [0x00007FF64FB9BDC7+1351783]
	GetHandleVerifier [0x00007FF64FB92A13+1313971]
	GetHandleVerifier [0x00007FF64FA8DD16+245686]
	(

In [9]:
## DATAFRAME CONVERSION

In [63]:
import pandas as pd

In [65]:
andhra = pd.DataFrame(APSRTC)
andhra

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3573,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),15:30,05h 55m,21:25,4.0,N/A,N/A
1,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 4052,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:30,06h 30m,22:00,3.8,N/A,N/A
2,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35271,INDRA(A.C. Seater),16:00,05h 50m,21:50,3.3,N/A,N/A
3,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3585,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:30,05h 55m,22:25,4.4,N/A,N/A
4,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.4,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
1534,Anantapur (andhra pradesh) to Hyderabad Bus,https://www.redbus.in/bus-tickets/ananthapur-t...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:35,06h 30m,06:05,4.4,N/A,11 Seats available
1535,Anantapur (andhra pradesh) to Hyderabad Bus,https://www.redbus.in/bus-tickets/ananthapur-t...,Sri Balaji Transports,A/C Sleeper (2+1),22:55,06h 27m,05:22,4.5,N/A,5 Seats available
1536,Anantapur (andhra pradesh) to Hyderabad Bus,https://www.redbus.in/bus-tickets/ananthapur-t...,Meena tours and travels,A/C Sleeper (2+1),22:05,07h 40m,05:45,4.3,N/A,6 Seats available
1537,Anantapur (andhra pradesh) to Hyderabad Bus,https://www.redbus.in/bus-tickets/ananthapur-t...,AdAl madeena Travels,A/C Sleeper (2+1),22:00,07h 40m,05:40,3.0,N/A,23 Seats available


In [77]:
kerala = pd.DataFrame(KSRTC)
kerala

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,09h 20m,04:24,4.4,N/A,20 Seats available
1,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,08h 25m,04:25,3.8,N/A,10 Seats available
2,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1585,Super Express Non AC Seater Air Bus (2+2),20:31,09h 05m,05:36,4.1,N/A,14 Seats available
3,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:31,09h 05m,06:36,3.4,N/A,4 Seats available
4,Bangalore to Kozhikode Bus,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,4.2,N/A,6 Seats available
...,...,...,...,...,...,...,...,...,...,...
429,Kalpetta (kerala) to Kozhikode Bus,https://www.redbus.in/bus-tickets/kalpetta-to-...,Cochin Express,A/C Seater Push Back (2+2),22:30,01h 25m,23:55,4.1,N/A,N/A
430,Kalpetta (kerala) to Kozhikode Bus,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Semi Sleeper (2+2),21:30,02h 10m,23:40,3.8,N/A,N/A
431,Kalpetta (kerala) to Kozhikode Bus,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),21:30,01h 59m,23:29,3.8,N/A,N/A
432,Kalpetta (kerala) to Kozhikode Bus,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:30,03h 59m,23:29,3.8,N/A,N/A


In [79]:
rajasthan = pd.DataFrame(RSRTC)
rajasthan

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 189473,NON AC Seater / Sleeper 2+1,21:00,04h 30m,01:30,4.0,N/A,24 Seats available
1,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 192299,Super Luxury Volvo AC Seater Pushback 2+2,23:00,03h 45m,02:45,3.8,N/A,29 Seats available
2,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 189583,NON AC Seater / Sleeper 2+1,23:59,03h 46m,03:45,4.1,N/A,15 Seats available
3,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jain travels regd,NON AC Seater / Sleeper 2+1,19:55,03h 55m,23:50,3.1,N/A,N/A
4,Jodhpur to Ajmer Bus,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jain travels regd,AC Sleeper (2+1),20:00,04h 30m,00:30,3.1,N/A,35 Seats available
...,...,...,...,...,...,...,...,...,...,...
526,Bikaner to Sikar Bus,https://www.redbus.in/bus-tickets/bikaner-to-s...,RATHORE TRAVELS AGENCY,A/C Semi Sleeper (2+2),05:00,03h 10m,08:10,3.5,N/A,N/A
527,Bikaner to Sikar Bus,https://www.redbus.in/bus-tickets/bikaner-to-s...,Milan Travels,Bharat Benz A/C Seater /Sleeper (2+1),05:00,03h 30m,08:30,3.9,N/A,N/A
528,Bikaner to Sikar Bus,https://www.redbus.in/bus-tickets/bikaner-to-s...,New Deep Travels,Volvo Multi-Axle I-Shift A/C Semi Sleeper (2+2),06:00,03h 10m,09:10,5.0,N/A,N/A
529,Bikaner to Sikar Bus,https://www.redbus.in/bus-tickets/bikaner-to-s...,Milan Travels,A/C Seater / Sleeper (2+1),08:10,03h 16m,11:26,4.1,N/A,N/A


In [81]:
telangana = pd.DataFrame(TSRTC)
telangana

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Khammam to Hyderabad Bus,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:30,04h 00m,21:30,3.8,N/A,N/A
1,Khammam to Hyderabad Bus,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,04h 00m,22:00,3.1,N/A,N/A
2,Khammam to Hyderabad Bus,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 1952,LAHARI A/C SLEEPER CUM SEATER,18:10,03h 35m,21:45,3.1,N/A,N/A
3,Khammam to Hyderabad Bus,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 4172,RAJADHANI AC (CONVERTED METRO LUXURY),18:30,04h 00m,22:30,3.7,N/A,N/A
4,Khammam to Hyderabad Bus,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8677,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",19:30,04h 10m,23:40,3.9,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
504,Hyderabad to Tirupati Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Zingbus Plus,A/C Sleeper (2+1),18:30,13h 25m,07:55,4.5,N/A,18 Seats available
505,Hyderabad to Tirupati Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Karthik Travels,Non A/C Seater / Sleeper (2+1),21:40,11h 35m,09:15,4.4,N/A,18 Seats available
506,Hyderabad to Tirupati Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,Moksha Travels,A/C Sleeper (2+1),19:05,11h 55m,07:00,4.1,N/A,13 Seats available
507,Hyderabad to Tirupati Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,AdBalaji Cabs,Bharat Benz A/C Sleeper (2+1),17:50,09h 40m,03:30,3.9,N/A,7 Seats available


In [69]:
kadamba = pd.DataFrame(KTCL)
kadamba

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,20:00,10h 21m,06:21,3.9,N/A,31 Seats available
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Non A/C Seater / Sleeper (2+1),19:30,12h 15m,07:45,4.3,N/A,29 Seats available
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Laxmi-Princess Nova,Non A/C Seater / Sleeper (2+1),19:30,11h 45m,07:15,4.0,N/A,14 Seats available
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray National Express,NON A/C Sleeper (2+1),20:00,10h 45m,06:45,4.1,N/A,16 Seats available
4,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Divine,NON A/C Sleeper (1+2),19:45,11h 00m,06:45,4.2,N/A,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
93,Goa to Sangli Bus,https://www.redbus.in/bus-tickets/goa-to-sangli,SR Tours & Travels,A/C Sleeper (2+1),20:00,07h 05m,03:05,2.7,N/A,16 Seats available
94,Goa to Miraj Bus,https://www.redbus.in/bus-tickets/goa-to-miraj,SR Tours & Travels,A/C Sleeper (2+1),20:00,07h 20m,03:20,2.7,N/A,16 Seats available
95,Shivamogga to Goa Bus,https://www.redbus.in/bus-tickets/shimoga-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater (2+2),23:00,09h 00m,08:00,2.6,N/A,38 Seats available
96,Shivamogga to Goa Bus,https://www.redbus.in/bus-tickets/shimoga-to-goa,Sea Bird Tourist,NON A/C Sleeper (2+1),23:45,08h 15m,08:00,4.2,N/A,11 Seats available


In [89]:
northbengal = pd.DataFrame(NBSTC)
northbengal

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Kolkata to Siliguri Bus,https://www.redbus.in/bus-tickets/kolkata-to-s...,NORTH BENGAL STATE TRANSPORT CORPORATION,Non AC Seater (2+3),15:00,13h 20m,04:20,3.3,N/A,49 Seats available
1,Kolkata to Siliguri Bus,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-R1-17:00,Non AC Seater 2+2,17:00,12h 00m,05:00,3.6,N/A,20 Seats available
2,Kolkata to Siliguri Bus,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-SLG-R1-18:00,NON A/C Seater Push Back (2+2),18:00,12h 00m,06:00,4.0,N/A,20 Seats available
3,Kolkata to Siliguri Bus,https://www.redbus.in/bus-tickets/kolkata-to-s...,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:45,12h 40m,07:25,4.7,N/A,1 Seat available
4,Kolkata to Siliguri Bus,https://www.redbus.in/bus-tickets/kolkata-to-s...,Lokenath Bus Service,Volvo Multi-Axle A/C Seater/Sleeper (2+1),18:15,12h 15m,06:30,4.5,N/A,43 Seats available
...,...,...,...,...,...,...,...,...,...,...
342,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,Lokenath Bus Service,NON A/C Seater / Sleeper (2+2),19:45,08h 25m,04:10,2.7,N/A,12 Seats available
343,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),19:15,07h 55m,03:10,2.4,N/A,33 Seats available
344,Cooch Behar (West Bengal) to Farakka Bus,https://www.redbus.in/bus-tickets/cooch-behar-...,NBSTC-COB-BHP-R1-19:00,Non AC Seater 2+2,19:00,09h 30m,04:30,4.7,N/A,28 Seats available
345,Cooch Behar (West Bengal) to Siliguri Bus,https://www.redbus.in/bus-tickets/cooch-behar-...,NBSTC-COB-FKT-SLG-06-15:30,Non AC Seater 2+2,15:30,03h 30m,19:00,2.3,N/A,N/A


In [91]:
bihar = pd.DataFrame(BSRTC)
bihar

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Patna to Bettiah Bus,https://www.redbus.in/bus-tickets/patna-to-bet...,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),06:00,05h 00m,11:00,3.8,N/A,N/A
1,Patna to Bettiah Bus,https://www.redbus.in/bus-tickets/patna-to-bet...,Bihar state road transport corporation (BSRTC)...,AC Seater (2+2),06:30,06h 05m,12:35,3.2,N/A,N/A
2,Patna to Bettiah Bus,https://www.redbus.in/bus-tickets/patna-to-bet...,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,3.0,N/A,60 Seats available
3,Patna to Bettiah Bus,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.4,N/A,54 Seats available
4,Patna to Bettiah Bus,https://www.redbus.in/bus-tickets/patna-to-bet...,Saurabh Travels,A/C Seater Push Back (2+2),07:00,05h 30m,12:30,1.0,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
177,Lucknow to Gopalganj (Bihar) Bus,https://www.redbus.in/bus-tickets/lucknow-to-g...,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,23:00,08h 00m,07:00,3.7,N/A,41 Seats available
178,Lucknow to Gopalganj (Bihar) Bus,https://www.redbus.in/bus-tickets/lucknow-to-g...,Samay Shatabdi Travels Pvt Ltd,AC Sleeper (2+1),21:00,06h 45m,03:45,4.4,N/A,30 Seats available
179,Lucknow to Gopalganj (Bihar) Bus,https://www.redbus.in/bus-tickets/lucknow-to-g...,MISHRA BANDHU BUS SERVICE,A/C Seater / Sleeper (2+1),21:30,07h 30m,05:00,3.8,N/A,44 Seats available
180,Lucknow to Gopalganj (Bihar) Bus,https://www.redbus.in/bus-tickets/lucknow-to-g...,Panwar Travels,A/C Sleeper (2+1),01:20,11h 40m,13:00,2.7,N/A,N/A


In [85]:
punjab = pd.DataFrame(PEPSU)
punjab

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66695,Volvo AC Seater (2+2),04:20,05h 01m,09:21,4.0,N/A,N/A
1,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66646,Volvo AC Seater (2+2),05:00,05h 01m,10:01,4.1,N/A,N/A
2,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66647,Volvo AC Seater (2+2),06:15,05h 01m,11:16,4.1,N/A,N/A
3,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67139,AC Seater Hvac 2+2,09:05,05h 01m,14:06,3.5,N/A,N/A
4,Patiala to Delhi Bus,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66648,Volvo AC Seater (2+2),09:50,05h 01m,14:51,4.1,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
523,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),12:00,01h 30m,13:30,2.4,N/A,N/A
524,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),13:30,01h 50m,15:20,2.4,N/A,N/A
525,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),13:55,01h 45m,15:40,2.5,N/A,N/A
526,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),15:25,01h 20m,16:45,2.2,N/A,N/A


In [87]:
westbengal = pd.DataFrame(WBSTC)
westbengal

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Kolkata to Digha Bus,https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-GARIA - DIGHA - 04:00 (DIGHA DEPOT) - 911,Non AC Seater (2+3),04:00,20h 00m,00:00,3.6,N/A,7 Seats available
1,Kolkata to Digha Bus,https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-GARIA - DIGHA - 04:30 (DIGHA DEPOT) - 1216,Non AC Seater (2+3),04:30,19h 30m,00:00,3.3,N/A,43 Seats available
2,Kolkata to Digha Bus,https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-GARIA - DIGHA - 05:00 (DIGHA DEPOT) - 910,Non AC Seater (2+3),05:00,19h 00m,00:00,3.2,N/A,12 Seats available
3,Kolkata to Digha Bus,https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-BELGHARIA - DIGHA - 05:15 (BEL DEPOT) - ...,Non AC Seater (2+3),05:15,04h 30m,09:45,3.5,N/A,N/A
4,Kolkata to Digha Bus,https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-GARIA - DIGHA - 05:30 (FALTA DEPOT) - 2996,Non AC Seater (2+3),05:30,18h 30m,00:00,2.9,N/A,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
300,Kolkata to Mandarmani Bus,https://www.redbus.in/bus-tickets/kolkata-to-m...,Suvam Travels,A/C Seater Push Back (2+3),15:30,03h 45m,19:15,1.8,N/A,N/A
301,Kolkata to Mandarmani Bus,https://www.redbus.in/bus-tickets/kolkata-to-m...,Anadi Travels (Disha Gold),A/C Seater Push Back (2+3),23:00,05h 30m,04:30,1.0,N/A,60 Seats available
302,Kolkata to Mandarmani Bus,https://www.redbus.in/bus-tickets/kolkata-to-m...,Aniruddha Travels,NON A/C Seater Push Back (2+3) (2+3),07:30,04h 50m,12:20,1.1,N/A,N/A
303,Kolkata to Bakkhali Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,WBTC (CTC) Kolkata - Namkhana ferry ghat - Bak...,Volvo AC Seater (2+2),06:30,03h 15m,09:45,4.2,N/A,N/A


In [75]:
jk = pd.DataFrame(JKSRTC)
jk

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,JKRTC - 169877,A/C Seater Executive (2+1),07:30,08h 00m,15:30,4.1,N/A,N/A
1,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,1.4,N/A,36 Seats available
2,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,Harikesh Tour N Travels,NON A/C Seater / Sleeper (2+2),20:30,09h 30m,06:00,2.4,N/A,44 Seats available
3,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,Northern Travels,Bharat Benz A/C Sleeper (2+1),07:00,07h 00m,14:00,2.9,N/A,N/A
4,Jammu to Srinagar (J and K) Bus,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),16:30,10h 00m,02:30,1.0,N/A,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
205,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,N/A,N/A
206,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,N/A,N/A
207,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),17:15,04h 35m,21:50,2.2,N/A,N/A
208,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),18:50,04h 50m,23:40,2.2,N/A,N/A


In [45]:
## CONCAT DATAFRAME

In [93]:
import pandas as pd

In [274]:
df = pd.concat([andhra,kerala,telangana,kadamba,rajasthan,punjab,northbengal,bihar,westbengal,jk])
df

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3573,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),15:30,05h 55m,21:25,4.0,N/A,N/A
1,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 4052,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:30,06h 30m,22:00,3.8,N/A,N/A
2,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35271,INDRA(A.C. Seater),16:00,05h 50m,21:50,3.3,N/A,N/A
3,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3585,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:30,05h 55m,22:25,4.4,N/A,N/A
4,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.4,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
205,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,N/A,N/A
206,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,N/A,N/A
207,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),17:15,04h 35m,21:50,2.2,N/A,N/A
208,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),18:50,04h 50m,23:40,2.2,N/A,N/A


## SQL CONNECT

In [276]:
## Creating Table

import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('bus_booking.db')

# Create a cursor object
cursor = conn.cursor()

# SQL statement to create the table
create_table_query = '''
CREATE TABLE IF NOT EXISTS bus_routes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    bus_route_name TEXT,
    bus_route_link TEXT,
    bus_name TEXT,
    bus_type TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL,
    seat_availability INTEGER
);
'''

# Execute the SQL statement
cursor.execute(create_table_query)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Table created successfully.")


Table created successfully.


In [278]:
## Viewing the coloumn values

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('bus_booking.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a PRAGMA statement to get the table schema
cursor.execute("PRAGMA table_info(bus_routes);")

# Fetch all results from the executed query
table_info = cursor.fetchall()

# Close the connection
conn.close()

# Print the structure of the table
for column in table_info:
    print(column)


(0, 'id', 'INTEGER', 0, None, 1)
(1, 'bus_route_name', 'TEXT', 0, None, 0)
(2, 'bus_route_link', 'TEXT', 0, None, 0)
(3, 'bus_name', 'TEXT', 0, None, 0)
(4, 'bus_type', 'TEXT', 0, None, 0)
(5, 'departing_time', 'TIME', 0, None, 0)
(6, 'duration', 'TEXT', 0, None, 0)
(7, 'reaching_time', 'TIME', 0, None, 0)
(8, 'star_rating', 'FLOAT', 0, None, 0)
(9, 'price', 'DECIMAL', 0, None, 0)
(10, 'seat_availability', 'INTEGER', 0, None, 0)


In [280]:
## Inserting Row values to the table

import sqlite3
import pandas as pd

# Example DataFrame (assuming df is already loaded with your data)
# df = pd.read_csv('your_data.csv')  # Uncomment if loading from a CSV file

# Connect to the SQLite database
conn = sqlite3.connect('bus_booking.db')

# Insert the DataFrame rows into the SQL table
df.to_sql('bus_routes', conn, if_exists='append', index=False)

# Close the connection
conn.close()

print("Data inserted successfully.")


Data inserted successfully.


In [282]:
# Viewing the created table after datas are inserted

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('bus_booking.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a SELECT statement to fetch all rows from the table
cursor.execute("SELECT * FROM bus_routes;")

# Fetch all results from the executed query
rows = cursor.fetchall()

# Close the connection
conn.close()

# Print the data in the table
for row in rows:
    print(row)


(1, 'Vijayawada to Hyderabad Bus', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 3573', 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', '15:30', '05h 55m', '21:25', 4.0, 'N/A', 'N/A')
(2, 'Vijayawada to Hyderabad Bus', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 4052', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', '15:30', '06h 30m', '22:00', 3.8, 'N/A', 'N/A')
(3, 'Vijayawada to Hyderabad Bus', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 35271', 'INDRA(A.C. Seater)', '16:00', '05h 50m', '21:50', 3.3, 'N/A', 'N/A')
(4, 'Vijayawada to Hyderabad Bus', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 3585', 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', '16:30', '05h 55m', '22:25', 4.4, 'N/A', 'N/A')
(5, 'Vijayawada to Hyderabad Bus', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'APSRTC - 35133', 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', '17:00', '05h 50m', '22:50', 3.4, 'N/A'

## CREATING APPLICATION USING STREAMLIT

In [284]:
import streamlit as st
import sqlite3
import pandas as pd

# Function to connect to the SQLite database
def get_connection():
    conn = sqlite3.connect('bus_booking.db')
    return conn

# Function to fetch bus routes based on user input
def search_bus_routes(departure_time=None, bus_type=None, max_price=None):
    conn = get_connection()
    query = "SELECT * FROM bus_routes WHERE 1=1"
    
    if departure_time:
        query += f" AND departing_time = '{departure_time}'"
    if bus_type:
        query += f" AND bus_type = '{bus_type}'"
    if max_price:
        query += f" AND price <= {max_price}"
    
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# Function to display the bus routes
def display_bus_routes(df):
    if df.empty:
        st.write("No bus routes found.")
    else:
        st.write("Available Bus Routes:")
        st.dataframe(df)

# Streamlit app
st.title("Bus Booking Application")

# Search criteria inputs
departure_time = st.time_input("Select Departure Time (optional):")
bus_type = st.selectbox("Select Bus Type (optional):", ["Any", "AC", "Non-AC"])
max_price = st.number_input("Enter Maximum Price (optional):", min_value=0.0, format="%.2f")

# Adjust bus type filter for SQL query
if bus_type == "Any":
    bus_type = None

# Button to search bus routes
if st.button("Search Buses"):
    bus_routes = search_bus_routes(departure_time, bus_type, max_price)
    display_bus_routes(bus_routes)

# Booking functionality (simple demonstration)
st.subheader("Book a Ticket")
selected_bus_id = st.number_input("Enter Bus ID to Book:", min_value=1)
if st.button("Book Ticket"):
    # Here you would typically add booking logic
    st.success(f"Ticket successfully booked for Bus ID {selected_bus_id}!")



In [1]:
pip install streamlit ipywidgets

In [3]:
import subprocess
import ipywidgets as widgets
from IPython.display import display, IFrame

# Path to your Streamlit app
streamlit_app_path = 'bus_booking_app.py'

# Run Streamlit app in a subprocess
def run_streamlit():
    subprocess.Popen(['streamlit', 'run', streamlit_app_path])

# Run the Streamlit app
run_streamlit()

# Display the Streamlit app in an iframe
display(IFrame(src="http://localhost:8501", width=800, height=600))


In [19]:
pip install mysql-connector-python

In [31]:
import mysql.connector

# Replace these values with your database connection details
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',  # or your host
    'database': 'project'
}

# Establish a connection to the database
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# SQL statement to create the table
create_table_query = """
CREATE TABLE bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    bus_route_name TEXT,
    bus_route_link TEXT,
    bus_name TEXT,
    bus_type TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seat_availability INT
);
"""

# Execute the SQL statement
cursor.execute(create_table_query)

print("Table created successfully.")

# Close the connection
cursor.close()
conn.close()


Table created successfully.


In [37]:
df['price'] = df['price'].replace('N/A', 0).infer_objects(copy=False)

In [41]:
df['seat_availability'] = df['seat_availability'].replace('N/A', 0)

In [45]:
df['seat_availability'] = df['seat_availability'].replace('N/A', -1)

In [77]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Assuming df is your DataFrame with 4683 rows

# Clean the 'star_rating' column
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce').fillna(0)
df = df[(df['star_rating'] >= 0) & (df['star_rating'] <= 5)]

# Replace these values with your database connection details
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',  # or your host
    'database': 'project'
}

try:
    # Establish a connection to the database
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # SQL statement to insert data
    insert_query = """
    INSERT INTO bus_routes (
        bus_route_name,
        bus_route_link,
        bus_name,
        bus_type,
        departing_time,
        duration,
        reaching_time,
        star_rating,
        price,
        seat_availability
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Convert DataFrame to a list of tuples
    data_tuples = [tuple(row) for row in df.to_numpy()]

    # Execute batch insert
    cursor.executemany(insert_query, data_tuples)

    # Commit the transaction
    conn.commit()

    print(f"{cursor.rowcount} rows inserted successfully.")

except Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


4683 rows inserted successfully.


In [79]:
df

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3573,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),15:30,05h 55m,21:25,4.0,0,0
1,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 4052,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:30,06h 30m,22:00,3.8,0,0
2,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35271,INDRA(A.C. Seater),16:00,05h 50m,21:50,3.3,0,0
3,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3585,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:30,05h 55m,22:25,4.4,0,0
4,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.4,0,0
...,...,...,...,...,...,...,...,...,...,...
4678,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,0,0
4679,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,0,0
4680,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),17:15,04h 35m,21:50,2.2,0,0
4681,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),18:50,04h 50m,23:40,2.2,0,0


In [69]:
print(df.iloc[7])  # Python uses 0-based indexing, so row 8 is index 7

bus_route_name                             Vijayawada to Hyderabad Bus
bus_route_link       https://www.redbus.in/bus-tickets/vijayawada-t...
bus_name                                                APSRTC - 35085
bus_type                DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)
departing_time                                                   18:30
duration                                                       06h 00m
reaching_time                                                    00:30
star_rating                                                        4.0
price                                                                0
seat_availability                                   43 Seats available
Name: 7, dtype: object


In [75]:
# Inspect the problematic row in the DataFrame
print(df.loc[1420, 'star_rating'])  # Indexing starts from 0, so row 1421 is 1420 in pandas


N/A


In [55]:
df

,bus_route_name,bus_route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability
0,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3573,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),15:30,05h 55m,21:25,4.0,0,0
1,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 4052,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:30,06h 30m,22:00,3.8,0,0
2,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35271,INDRA(A.C. Seater),16:00,05h 50m,21:50,3.3,0,0
3,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3585,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:30,05h 55m,22:25,4.4,0,0
4,Vijayawada to Hyderabad Bus,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35133,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),17:00,05h 50m,22:50,3.4,0,0
...,...,...,...,...,...,...,...,...,...,...
4678,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,0,0
4679,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,0,0
4680,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),17:15,04h 35m,21:50,2.2,0,0
4681,Jammu to Amritsar Bus,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),18:50,04h 50m,23:40,2.2,0,0


In [110]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming `df` is your DataFrame
# Replace these with your actual MySQL credentials and database information
username = 'root'
password = 'root'
host = 'localhost'
database = 'dt28'

# Create a connection to the database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# Insert the DataFrame into the SQL table
df.to_sql('bus_routes', con=engine, if_exists='append', index=False)


4683

In [156]:
import mysql.connector

# Connect to your MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="dt28"
)

cursor = connection.cursor()

# Execute the query to count the rows
cursor.execute("SELECT COUNT(*) FROM dt28.bus_routes")

# Fetch the result
row_count = cursor.fetchone()[0]

# Display the result
print(f"Total number of rows in the bus_routes table: {row_count}")

# Close the cursor and connection
cursor.close()
connection.close()


Total number of rows in the bus_routes table: 4683


In [160]:
print(df.columns)


Index(['Id', 'bus_route_name', 'bus_route_link', 'bus_name', 'bus_type',
       'departing_time', 'duration', 'reaching_time', 'star_rating', 'price',
       'seat_availability'],
      dtype='object')
